In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import random
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [11]:
# source directory
image_dir = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\combined_images\VeryMildDemented"
image_dir1 = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\combined_images\NonDemented" 
image_dir2 = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\combined_images\ModerateDemented"
image_dir3 = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\combined_images\MildDemented"


# destination directory
image_dir_copy = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\VeryMildDemented_1200_copy"
image_dir1_copy = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\NonDemented_1200_copy" 
image_dir2_copy= r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\ModerateDemented_1200_copy"
image_dir3_copy = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\MildDemented_1200_copy"


In [12]:
# List image files and sort them
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png", ".jpeg"))])

# Show first 5 images
for img_file in image_files[:5]:
    img_path = os.path.join(image_dir, img_file)
    image = Image.open(img_path)
    image.show()  # Opens the image
    print(f"Displayed: {img_file}")

Displayed: 0001b959-d622-4311-acab-84633370c892.jpg
Displayed: 0003659d-f8db-4ce4-9230-2ba24506df68.jpg
Displayed: 000a074f-a3a5-4c70-8c94-d7ed7bbe7018.jpg
Displayed: 000b7abc-2404-411d-a46d-467ec55b7795.jpg
Displayed: 000dea20-ea76-4248-a45d-4119f0bc5ccc.jpg


In [13]:
# number of images in a directory
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png", ".jpeg"))])
image_files1 = sorted([f for f in os.listdir(image_dir1) if f.endswith((".jpg", ".png", ".jpeg"))])
image_files2 = sorted([f for f in os.listdir(image_dir2) if f.endswith((".jpg", ".png", ".jpeg"))])
image_files3 = sorted([f for f in os.listdir(image_dir3) if f.endswith((".jpg", ".png", ".jpeg"))])

In [14]:
# for the sake of no sufficient resources we will take a sample each of 1200 images

# Ensure the destination directory exists
os.makedirs(image_dir_copy, exist_ok=True)
os.makedirs(image_dir1_copy, exist_ok=True)
os.makedirs(image_dir2_copy, exist_ok=True)
os.makedirs(image_dir3_copy, exist_ok=True)

In [15]:
# Get all image files

# random.seed(42) 

# Select 1200 random images (same selection every time)
selected_images = random.sample(image_files, min(1200, len(image_files1)))
selected_images1 = random.sample(image_files1, min(1200, len(image_files1)))
selected_images2 = random.sample(image_files2, min(1200, len(image_files2)))
selected_images3 = random.sample(image_files3, min(1200, len(image_files3)))

# Copy selected images to the new directory
for img in selected_images:
    shutil.copy(os.path.join(image_dir, img), os.path.join(image_dir_copy, img))
for img in selected_images1:
    shutil.copy(os.path.join(image_dir1, img), os.path.join(image_dir1_copy, img))
for img in selected_images2:
    shutil.copy(os.path.join(image_dir2, img), os.path.join(image_dir2_copy, img))
for img in selected_images3:
    shutil.copy(os.path.join(image_dir3, img), os.path.join(image_dir3_copy, img))

print(f"Selected {len(selected_images)} images and copied them to {image_dir_copy}.")
print(f"Selected {len(selected_images1)} images and copied them to {image_dir1_copy}.")
print(f"Selected {len(selected_images2)} images and copied them to {image_dir2_copy}.")
print(f"Selected {len(selected_images3)} images and copied them to {image_dir3_copy}.")

Selected 1200 images and copied them to C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\VeryMildDemented_1200_copy.
Selected 1200 images and copied them to C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\NonDemented_1200_copy.
Selected 1200 images and copied them to C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\ModerateDemented_1200_copy.
Selected 1200 images and copied them to C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\MildDemented_1200_copy.


In [17]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [18]:
# Preprocessing & Data Augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.RandomHorizontalFlip(),  # Augment data
    transforms.RandomRotation(10),
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

In [20]:

# Load dataset (Modify path as needed)
dataset_path = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Alzhermiers Classification\dataset"  # Change to your dataset directory
train_dataset = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{dataset_path}/val", transform=transform)


In [21]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [26]:
# Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Adjust for image size
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.bn1(self.conv1(x))))
        x = self.pool(nn.ReLU()(self.bn2(self.conv2(x))))
        x = self.pool(nn.ReLU()(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)  # Flatten for FC layers
        x = nn.ReLU()(self.fc1(x))
        # x = self.dropout(x)
        x = self.fc2(x)
        return x

# Model Initialization
model = CustomCNN(num_classes=4).to(device)

In [27]:
# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)

# Training Function
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")


In [28]:
# Evaluation Function
def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total


In [29]:
# Train the Model
train_model(model, train_loader, val_loader, epochs=10)

Epoch 1/10, Loss: 254.9478, Train Acc: 0.3944, Val Acc: 0.4752
Epoch 2/10, Loss: 147.8894, Train Acc: 0.5531, Val Acc: 0.6138
Epoch 3/10, Loss: 124.4842, Train Acc: 0.6238, Val Acc: 0.6358
Epoch 4/10, Loss: 114.2363, Train Acc: 0.6585, Val Acc: 0.5846
Epoch 5/10, Loss: 108.0062, Train Acc: 0.6727, Val Acc: 0.6421
Epoch 6/10, Loss: 99.7400, Train Acc: 0.7002, Val Acc: 0.7063
Epoch 7/10, Loss: 96.3553, Train Acc: 0.7115, Val Acc: 0.6110
Epoch 8/10, Loss: 91.2330, Train Acc: 0.7273, Val Acc: 0.7044
Epoch 9/10, Loss: 89.3631, Train Acc: 0.7367, Val Acc: 0.6965
Epoch 10/10, Loss: 87.4135, Train Acc: 0.7425, Val Acc: 0.7342


In [ ]:
# evaluate_model(model, loader)